# Memory
## Memory 是什麼？
Memory 讓 LangGraph 能夠記住對話歷史和 graph 的執行狀態，實現：

- 狀態持久化：保存每次執行的 state
- 對話延續：透過 thread_id 區分不同對話
- 歷史追蹤：查看 graph 在每個節點的狀態變化
- Time Travel：回到過去任意時間點繼續執行

## Checkpointer
LangGraph 使用 Checkpointer 來管理 Memory。每次 graph 執行完一個節點，就會保存當下的 state。

## MemorySaver（記憶體存儲）
MemorySaver 是 LangGraph 提供的 最簡單 checkpoint 實作，適合用於 本地測試、教學、開發階段。
它的功能是：
> 在每次 graph 執行結束後，將 state 儲存在記憶體中，並以 thread_id 作為索引

### 基本範例

In [1]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from typing import TypedDict

class AgentState(TypedDict):
    messages: list[str]
    count: int

def process_node(state: AgentState):
    new_message = f"處理第 {state['count']} 次"
    return {
        "messages": state["messages"] + [new_message],
        "count": state["count"] + 1
    }

# 建立 graph
workflow = StateGraph(AgentState)
workflow.add_node("process", process_node)
workflow.set_entry_point("process")
workflow.set_finish_point("process")

# 使用 MemorySaver
checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "conversation_1"}}

說明：

- thread_id：對話識別碼，相同 thread_id 會共享狀態
- MemorySaver 只存在記憶體中，程式重啟後會消失

In [2]:
state = graph.invoke({"messages": [], "count": 1}, config)
print(state)
while True:
    state = graph.invoke(state, config)
    print(state)

    # 停止條件（任選）
    if state["count"] > 3:
        break

{'messages': ['處理第 1 次'], 'count': 2}
{'messages': ['處理第 1 次', '處理第 2 次'], 'count': 3}
{'messages': ['處理第 1 次', '處理第 2 次', '處理第 3 次'], 'count': 4}


### to be continued